<a href="https://www.kaggle.com/code/bsrsrc/bg-nbd-and-gamma-gamma-for-cltv-prediction?scriptVersionId=180260374" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# BG-NBD and Gamma-Gamma for CLTV Prediction

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
!pip install lifetimes
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
pd.set_option("display.float_format", lambda x: "%.4f" % x)
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from sklearn.preprocessing import MinMaxScaler


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00


## Business Problem

#### FLO wants to determine a roadmap for sales and marketing activities.

In order for the company to plan for the medium to long term, it is necessary to predict the potential value that existing customers will bring to the company in the future.

## Dataset Story

The dataset consists of information derived from the past shopping behavior of customers who made their last purchases OmniChannel (both online and offline) between 2020 and 2021.

master_id: Unique customer number

order_channel: Which channel used for shopping (Android, iOS, Desktop, Mobile, Offline)

last_order_channel: The channel where the last purchase was made

first_order_date: The date of the first purchase made by the customer

last_order_date: The date of the last purchase made by the customer

last_order_date_online: The date of the last purchase made on the online platform

last_order_date_offline: The date of the last purchase made on the offline platform

order_num_total_ever_online: Total number of purchases made by the customer on the online 
platform


order_num_total_ever_offline: Total number of purchases made by the customer on the offline platform

customer_value_total_ever_offline: Total amount paid by the customer in offline purchases

customer_value_total_ever_online: Total amount paid by the customer in online purchases

interested_in_categories_12: List of categories that the customer has made purchases in the last 12 months

## TASKS

### TASK 1: Data Preparation

1. Read the flo-data.csv data. Create a copy of the DataFrame.

In [2]:
df_ = pd.read_csv("/kaggle/input/flo-data/flo_data_20k.csv")
df = df_.copy()
df.columns
#quartile1 = df['order_num_total_ever_online'].quantile(0.01)
#quartile3 = df["order_num_total_ever_online"].quantile(0.99)
#quartile1
#quartile3
#df["order_num_total_ever_online"].max

Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date',
       'last_order_date', 'last_order_date_online', 'last_order_date_offline',
       'order_num_total_ever_online', 'order_num_total_ever_offline',
       'customer_value_total_ever_offline', 'customer_value_total_ever_online',
       'interested_in_categories_12'],
      dtype='object')

## 2. Define the functions outlier_thresholds and replace_with_thresholds required to suppress outliers.

#### Note: When calculating CLTV, frequency values must be integers. Therefore, round their lower and upper limits with round().

In [3]:
def outlier_thresholds(df, variable):
    quartile1 = df[variable].quantile(0.01)
    quartile3 = df[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(df, variable):
    low_limit, up_limit = outlier_thresholds(df, variable)
    df.loc[(df[variable] > up_limit), variable] = round(up_limit)
    df.loc[(df[variable] < low_limit), variable] = round(low_limit)

#### Suppress outliers if there are any in the variables "order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline", "customer_value_total_ever_online".

In [4]:
replace_with_thresholds(df, "order_num_total_ever_online")
replace_with_thresholds(df, "order_num_total_ever_offline")
replace_with_thresholds(df, "customer_value_total_ever_offline")
replace_with_thresholds(df, "customer_value_total_ever_online")


### Omnichannel customers refer to customers who make purchases from both online and offline platforms.

#### Create new variables for each customer's total number of purchases and spending.

In [5]:
df["total_order_num"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["total_customer_value"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order_num,total_customer_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN],5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0000,2013.5500
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]",5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]",2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR],2.0000,209.9800


### Examine variable types. Convert variables representing dates to date type.

In [6]:
df.info()
date_cols = df.columns[df.columns.str.contains("date")]
df[date_cols] = df[date_cols].apply(lambda x: pd.to_datetime(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

## TASK 2: Creating the CLTV Data Structure

### Take the date two days after the last purchase date in the dataset as the analysis date.

In [7]:
df["last_order_date"].max()
today_date = dt.datetime(2021, 6, 1)

Timestamp('2021-05-30 00:00:00')

## Task 2: Creating a New CLTV DataFrame Containing customer_id, recency_cltv_weekly, T_weekly, frequency, and monetary_cltv_avg values.

In [8]:
cltv = pd.DataFrame()
cltv["customer_id"] = df["master_id"]
cltv["recency_cltv_weekly"] = (df["last_order_date"] - df["first_order_date"]).dt.days / 7
cltv["T_weekly"] = (today_date - df["first_order_date"]).dt.days / 7
cltv["frequency"] = df["total_order_num"]
cltv["monetary_cltv_avg"] = df["total_customer_value"]
cltv.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,2013.5500
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,209.9800


## TASK 3: Establishing BG/NBD and Gamma-Gamma Models, Calculating 6-Month CLTV

### 1. Establish the BG/NBD model.

In [9]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv["frequency"],
        cltv["recency_cltv_weekly"],
        cltv["T_weekly"])

/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

#### Predict expected purchases from customers within 3 months and add it to the cltv DataFrame as exp_sales_3_month.

In [10]:
cltv["exp_sales_3_month"] = bgf.predict(4*3,
                                       cltv["frequency"],
                                       cltv["recency_cltv_weekly"],
                                       cltv["monetary_cltv_avg"]).sort_values(ascending=False)
cltv.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,939.3700,0.1024
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,2013.5500,0.1381
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,585.3200,0.1572
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,121.9700,0.3430
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,209.9800,0.2375


#### Predict expected purchases from customers within 6 months and add it to the cltv DataFrame as exp_sales_6_month.


In [11]:
cltv["exp_sales_6_month"] = bgf.predict(4*6,
                                       cltv["frequency"],
                                       cltv["recency_cltv_weekly"],
                                       cltv["monetary_cltv_avg"]).sort_values(ascending=False)
cltv.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,939.3700,0.1024,0.2047
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,2013.5500,0.1381,0.2762
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,585.3200,0.1572,0.3143
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,121.9700,0.3430,0.6860
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,209.9800,0.2375,0.4750


## Examine the top 10 customers with the most purchases in the 3rd and 6th months.

In [12]:
cltv.sort_values(by="exp_sales_3_month", ascending=False).head(10)
cltv.sort_values(by="exp_sales_6_month", ascending=False).head(10)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
1710,08b95b62-a987-11e9-a2fc-000d3a38a36f,226.5714,232.4286,2.0000,44.9800,0.5610,1.1219
14980,0769f8d0-d627-11e9-93bc-000d3a38a36f,58.8571,93.4286,2.0000,44.9800,0.5610,1.1219
9621,1b22a066-d68b-11e9-93bc-000d3a38a36f,45.8571,96.1429,3.0000,67.9700,0.5547,1.1095
7693,1e2dd6cc-0b1e-11eb-8ffc-000d3a38a36f,20.7143,33.4286,2.0000,49.9800,0.5387,1.0774
19341,0dd9a790-7d63-11ea-80af-000d3a38a36f,13.8571,59.1429,2.0000,53.4700,0.5242,1.0484
12666,1af373d6-acbf-11e9-a2fc-000d3a38a36f,76.2857,91.2857,2.0000,55.1800,0.5174,1.0348
5137,a5e9ca3e-d632-11e9-93bc-000d3a38a36f,46.1429,93.2857,2.0000,55.9800,0.5143,1.0285
3096,4bb3a040-acc9-11e9-a2fc-000d3a38a36f,136.7143,159.7143,3.0000,80.4700,0.5105,1.0209
16003,c83ae112-ddea-11e9-a848-000d3a38a36f,78.4286,96.4286,3.0000,80.8600,0.5092,1.0184
16048,6b539a62-544a-11ea-b1db-000d3a38a36f,47.5714,76.7143,3.0000,80.9700,0.5088,1.0177


,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
1710,08b95b62-a987-11e9-a2fc-000d3a38a36f,226.5714,232.4286,2.0000,44.9800,0.5610,1.1219
14980,0769f8d0-d627-11e9-93bc-000d3a38a36f,58.8571,93.4286,2.0000,44.9800,0.5610,1.1219
9621,1b22a066-d68b-11e9-93bc-000d3a38a36f,45.8571,96.1429,3.0000,67.9700,0.5547,1.1095
7693,1e2dd6cc-0b1e-11eb-8ffc-000d3a38a36f,20.7143,33.4286,2.0000,49.9800,0.5387,1.0774
19341,0dd9a790-7d63-11ea-80af-000d3a38a36f,13.8571,59.1429,2.0000,53.4700,0.5242,1.0484
12666,1af373d6-acbf-11e9-a2fc-000d3a38a36f,76.2857,91.2857,2.0000,55.1800,0.5174,1.0348
5137,a5e9ca3e-d632-11e9-93bc-000d3a38a36f,46.1429,93.2857,2.0000,55.9800,0.5143,1.0285
3096,4bb3a040-acc9-11e9-a2fc-000d3a38a36f,136.7143,159.7143,3.0000,80.4700,0.5105,1.0209
16003,c83ae112-ddea-11e9-a848-000d3a38a36f,78.4286,96.4286,3.0000,80.8600,0.5092,1.0184
16048,6b539a62-544a-11ea-b1db-000d3a38a36f,47.5714,76.7143,3.0000,80.9700,0.5088,1.0177


## Fit the Gamma-Gamma model. Predict the average value that customers will leave and add it to the cltv DataFrame as exp_average_value.


In [13]:
ggf = GammaGammaFitter(penalizer_coef=0.001)
ggf.fit(cltv["frequency"],
        cltv["monetary_cltv_avg"])
ggf.conditional_expected_average_profit(cltv["frequency"], cltv["monetary_cltv_avg"]).head()
ggf.conditional_expected_average_profit(cltv["frequency"], cltv["monetary_cltv_avg"]).sort_values(ascending=False).head()
cltv["exp_average_value"] = ggf.conditional_expected_average_profit(cltv["frequency"], cltv["monetary_cltv_avg"]).head()
cltv.sort_values(by="exp_average_value", ascending=False).head()

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 12.73, q: 0.69, v: 12.61>

0    946.5479
1   2016.5121
2    590.7478
3    129.8720
4    218.9777
dtype: float64

7613    9068.5690
7330    8648.0127
11150   8489.3980
13880   8358.7750
14181   8185.6468
dtype: float64

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,2013.5500,0.1381,0.2762,2016.5121
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,939.3700,0.1024,0.2047,946.5479
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,585.3200,0.1572,0.3143,590.7478
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,209.9800,0.2375,0.4750,218.9777
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,121.9700,0.3430,0.6860,129.8720


## Calculate the 6-month CLTV and add it to the DataFrame as cltv.

In [14]:
cltv["cltv"] = ggf.customer_lifetime_value(bgf,
                                          cltv["frequency"],
                                          cltv["recency_cltv_weekly"],
                                          cltv["T_weekly"],
                                          cltv["monetary_cltv_avg"],
                                          time=6,
                                          freq="W",
                                          discount_rate=0.01)
cltv.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,939.3700,0.1024,0.2047,946.5479,1934.4898
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,2013.5500,0.1381,0.2762,2016.5121,4160.2876
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,585.3200,0.1572,0.3143,590.7478,831.2932
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,121.9700,0.3430,0.6860,129.8720,190.8827
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,209.9800,0.2375,0.4750,218.9777,181.9848


## Observe the top 20 customers with the highest CLTV values.

In [15]:
cltv.sort_values(by="cltv", ascending=False).head(20)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.7143,67.2857,52.0000,8643.6800,0.0000,0.0000,NaN,84496.7210
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,133.1429,147.1429,49.0000,7930.4900,0.0000,0.0000,NaN,47119.5921
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.5714,58.7143,31.0000,7084.4300,0.0000,0.0000,NaN,45883.3868
14181,0e8c7b30-ac18-11e9-a2fc-000d3a38a36f,141.7143,159.7143,48.0000,8181.1900,0.0000,0.0000,NaN,45145.1838
10489,7af5cd16-b100-11e9-9757-000d3a38a36f,103.1429,111.8571,43.0000,6755.8400,0.0000,0.0000,NaN,42245.0506
10536,e143b6fa-d6f8-11e9-93bc-000d3a38a36f,104.5714,113.4286,40.0000,7048.0000,0.0000,0.0000,NaN,40898.5849
14330,90d628b6-1928-11ea-b6a2-000d3a38a36f,98.5714,117.4286,32.0000,7879.9800,0.0000,0.0000,NaN,36582.6177
4157,7eed6468-4540-11ea-acaf-000d3a38a36f,89.1429,90.0000,27.0000,7823.6300,0.0000,0.0000,NaN,36389.0086
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,204.8571,211.8571,49.0000,7882.4800,0.0000,0.0000,NaN,36311.6351
16335,f24f6988-acd9-11e9-a2fc-000d3a38a36f,172.8571,194.7143,57.0000,6275.6800,0.0000,0.0000,NaN,35406.3772


## TASK 4: Creating Segments Based on CLTV
####  Segment all your customers into 4 groups (segments) based on the 6-month CLTV and add segment names to the dataset.
#### Assign them as cltv_segment.

In [16]:
cltv["segment"] = pd.qcut(cltv["cltv"], 4, labels=["D", "C", "B", "A"])
cltv.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,939.3700,0.1024,0.2047,946.5479,1934.4898,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,2013.5500,0.1381,0.2762,2016.5121,4160.2876,A
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,585.3200,0.1572,0.3143,590.7478,831.2932,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,121.9700,0.3430,0.6860,129.8720,190.8827,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,209.9800,0.2375,0.4750,218.9777,181.9848,D


## Examine the recency, frequency, and monetary averages of the segments.

In [17]:
cltv.groupby("segment").agg({"count", "sum"})
cltv.describe().T

customer_id        \
                                                       sum count   
segment                                                            
D        1854e56c-491f-11eb-806e-000d3a38a36fd6ea1074-f...  4987   
C        c445e4ee-6242-11ea-9d1a-000d3a38a36f2854f11a-0...  4986   
B        69b69676-1a40-11ea-941b-000d3a38a36fec6c382a-3...  4986   
A        cc294636-19f0-11eb-8d74-000d3a38a36ff431bd5a-a...  4986   

        recency_cltv_weekly          T_weekly        frequency        \
                        sum count         sum count        sum count   
segment                                                                
D               514805.7143  4987 631156.1429  4987 12767.0000  4987   
C               419214.2857  4986 520955.7143  4986 16805.0000  4986   
B               439262.8571  4986 530770.1429  4986 23334.0000  4986   
A               526747.5714  4986 600258.7143  4986 46383.0000  4986   

        monetary_cltv_avg       exp_sales_3_month       exp_sales_6_month  \
                      sum count               sum count               sum   
segment                                                                     
D            1298744.6600  4987         1187.0554  4987         2374.1108   
C            2247647.8700  4986          830.3654  4986         1660.7308   
B            3505898.1200  4986          665.2784  4986         1330.5567   
A            7739459.7900  4986          482.6418  4986          965.2837   

              exp_average_value                cltv        
        count               sum count           sum count  
segment                                                    
D        4987          348.8497     2  1095504.5494  4987  
C        4986            0.0000     0  2283921.4021  4986  
B        4986          590.7478     1  4149905.0086  4986  
A        4986         2963.0600     2 14694666.7386  4986

,count,mean,std,min,25%,50%,75%,max
recency_cltv_weekly,19945.0000,95.2635,74.5894,0.0000,50.4286,76.5714,109.4286,433.4286
T_weekly,19945.0000,114.4718,74.7710,0.7143,73.8571,93.0000,119.4286,437.1429
frequency,19945.0000,4.9781,4.1209,2.0000,3.0000,4.0000,6.0000,57.0000
monetary_cltv_avg,19945.0000,741.6270,687.9057,44.9800,339.9800,545.2700,897.7800,9063.7600
exp_sales_3_month,19945.0000,0.1587,0.0695,0.0000,0.1113,0.1482,0.1957,0.5610
exp_sales_6_month,19945.0000,0.3174,0.1391,0.0000,0.2227,0.2965,0.3915,1.1219
exp_average_value,5.0000,780.5315,763.2050,129.8720,218.9777,590.7478,946.5479,2016.5121
cltv,19945.0000,1114.2641,2068.1043,23.9950,335.6431,597.7894,1159.1307,84496.7210
